In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!cp /content/drive/MyDrive/fake-news-explainability/utils_fake_news.py .

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4
import time
import datetime
import random
import itertools
import os

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import (
    Dataset, 
    DataLoader,
    TensorDataset, 
    random_split, 
    RandomSampler, 
    SequentialSampler)

# Transformers
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    AdamW,
    get_linear_schedule_with_warmup)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

%run utils_fake_news.py

# Device
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

## Model & Training Function

In [ ]:
import torch.nn as nn
from transformers import AutoModel
class FakeBERT(nn.Module):
    def __init__(self):
        super(FakeBERT, self).__init__()
        
        self.base_model = AutoModel.from_pretrained('bert-base-uncased')

        # Layer 1: Conv1D + Maxpool
        self.conv_1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1)
        self.sigm_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool1d(kernel_size=5, stride=5)

        # Layer 2: Conv1D + Maxpool
        self.conv_2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=1)
        self.sigm_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool1d(kernel_size=5, stride=5)

        # Layer 5: Fully Connected Layer 
        self.full_5 = nn.Linear(306,128)
        self.sigm_5 = nn.Sigmoid()

        # Layer 5: Fully Connected Layer 
        self.full_6 = nn.Linear(128,32)
        self.sigm_6 = nn.Sigmoid()
        
        # Layer 7: Fully Connected Layer 
        self.full_7 = nn.Linear(32,2)
        self.soft_7 = nn.Softmax()

    def forward(self, input_ids, attn_mask):
        bert_output = self.base_model(input_ids, attention_mask=attn_mask)
        bert_output = bert_output['pooler_output'].unsqueeze(1)

        output1 = self.pool_1(self.sigm_1(self.conv_1(bert_output))).squeeze(1)
        output2 = self.pool_2(self.sigm_2(self.conv_2(bert_output))).squeeze(1)
        outputs = torch.cat((output1,output2),dim=1)
        outputs = self.sigm_5(self.full_5(outputs))
        outputs = self.sigm_6(self.full_6(outputs))
        outputs = self.soft_7(self.full_7(outputs))
        return outputs

def train():
    total_t0 = time.time()
    for epoch_i in range(0, epochs):
        
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        t0 = time.time()
        total_train_loss = 0
        bert_model.train()

        for step, batch in enumerate(bert_train_dataloader):

            # Unpack batch
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Zero grads
            bert_model.zero_grad()        

            # Forward pass
            output = bert_model(b_input_ids, b_input_mask)
            
            # Accumulate loss
            loss = loss_func(output.squeeze(1), b_labels)

            # Backpropagate
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The bert_optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            bert_optimizer.step()

            # Update the learning rate.
            bert_scheduler.step()

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(bert_train_dataloader), elapsed))
                print(loss.detach())

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(bert_train_dataloader)            
        
        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

        # Record all statistics from this epoch.
        bert_training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                # 'Valid. Loss': avg_val_loss,
                # 'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                # 'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## Load Data

In [ ]:
# Load encoded Fake-News Kaggle dataset
df_train_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/training/fake_news.pt")

In [ ]:
# Load data into dataloader
batch_size = 32
bert_train_dataloader = DataLoader(
    df_train_encode,  # The training samples.
    batch_size = batch_size # Trains with this batch size.
    )

## Training

In [ ]:
# BERT
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Model
bert_model = FakeBERT().to(device)

In [ ]:
# Optimizer
bert_optimizer = AdamW(bert_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Training Params
bert_training_stats = []
epochs = 2
total_steps = len(bert_train_dataloader) * epochs
loss_func = nn.CrossEntropyLoss()

# Learning rate scheduler.
bert_scheduler = get_linear_schedule_with_warmup(bert_optimizer, 
                                                 num_warmup_steps = 0, # Default value in run_glue.py
                                                 num_training_steps = total_steps)

In [ ]:
# Train or load pre-trained
bert_model_path = "/content/drive/MyDrive/fake-news-explainability/Models/fake_news_model"

if os.path.exists(bert_model_path):
    bert_model = FakeBERT().to(device)
    bert_model.load_state_dict(torch.load(bert_model_path))
else:
    train()
    torch.save(bert_model.state_dict(), bert_model_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Evaluate

In [ ]:
# Load encoded tensors
df_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/training/fake_news.pt")

# Load data into dataloader
batch_size = 32
bert_dataloader = DataLoader(df_encode, batch_size = batch_size)

# Generate predictions
outputs = []
with torch.no_grad():
    for step, batch in enumerate(bert_dataloader):
        # Unpack batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Forward pass
        output = bert_model(b_input_ids, b_input_mask)
        outputs.append(output)        
        
# Stack outputs
outputs = torch.vstack(outputs)

# Accuracy
print(f"Acc: {flat_accuracy(outputs.detach().cpu().numpy(), df_encode.tensors[2].to('cpu').numpy())}")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Acc: 0.9101373382076869


## Adversarial Attack Evaluation

In [ ]:
# Load encoded tensors
# df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_pos.pt")
# df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_neg.pt")

df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_name_orig_filtered.pt")
df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_name_new_filtered.pt")

# df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_polarity_orig_filtered.pt")
# df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_polarity_new_filtered.pt")

# Load data into dataloader
batch_size = 32
bert_pos_dataloader = DataLoader(df_pos_encode, batch_size = batch_size)
bert_neg_dataloader = DataLoader(df_neg_encode, batch_size = batch_size)

In [ ]:
# Generate predictions
# Can't input everything directly due to RAM issues
outputs_pos = []
outputs_neg = []
with torch.no_grad():
    for step, batch in enumerate(bert_pos_dataloader):
        # Unpack batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Forward pass
        output = bert_model(b_input_ids, b_input_mask)
        outputs_pos.append(output)
        
    for step, batch in enumerate(bert_neg_dataloader):
        # Unpack batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Forward pass
        output = bert_model(b_input_ids, b_input_mask)
        outputs_neg.append(output)
        
# Stack outputs
outputs_pos = torch.vstack(outputs_pos)
outputs_neg = torch.vstack(outputs_neg)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
# Percent Labels Flipped
cf_matrix = confusion_matrix(torch.argmax(outputs_pos.cpu(), axis=1), 
                             torch.argmax(outputs_neg.cpu(), axis=1))
print(f"Labels Flipped: {cf_matrix[0,1]+cf_matrix[1,0]} of {np.sum(cf_matrix)} ({round(100*(cf_matrix[0,1]+cf_matrix[1,0])/np.sum(cf_matrix),4)}%)")

Labels Flipped: 1 of 248 (0.4032%)


In [ ]:
# Average Probability Change
m = nn.Softmax(dim=1)
delta_lst = m(outputs_neg)[:,1]-m(outputs_pos)[:,1]
print(f"Average Change: {round(float(torch.mean(delta_lst)),4)} ({round(float(torch.std(delta_lst)),4)})")

Average Change: 0.0015 (0.0234)
